<FONT size="5.9pt"> **NB1 - PROSPECTIVE CONSEQUENTIAL BACKGROUND DATABASES (ECOINVENT 3.9)**
</FONT> 


This jupyter notebook was used to generate the future background databases for the consequential prospective LCA study of bark combustion versus biorefinery using the [*premise*](https://github.com/polca/premise) Python library. The use of *premise* requires a unique **"decryption key"** to run the code that can be obtained by contacting the developer (see documentation). 

In [3]:
import os
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# define the initial working directory
wdir = Path().resolve() # get the directory of the notebook
pdir = os.path.join(wdir,'projects')
print(f"Working Directory: {wdir}")

# set the BW project directory
pathlib.Path(pdir).mkdir(parents=True, exist_ok=True)
os.environ['BRIGHTWAY2_DIR'] = pdir
print('Project directory:', os.environ['BRIGHTWAY2_DIR'])

# set the directory to store the results
result_dir = os.path.join(wdir,'results')   
pathlib.Path(result_dir).mkdir(parents=True, exist_ok=True)
print('Results directory:', result_dir)

# set the directory in which the databases are stored
db_dir = os.path.join(wdir,'databases')
pathlib.Path(db_dir).mkdir(parents=True, exist_ok=True)
print('Databases directory:', db_dir)

Working Directory: /home/leabraud/Documents/LCAbark
Project directory: /home/leabraud/Documents/LCAbark/projects
Results directory: /home/leabraud/Documents/LCAbark/results
Databases directory: /home/leabraud/Documents/LCAbark/databases


In [5]:
import lca_algebraic as alg
import brightway2 as bw

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/leabraud/Documents/LCAbark/projects


## Set up a project

In [6]:
# Create or select a project 
project_name = 'lca_bark'
bw.projects.set_current(project_name)

if project_name in bw.projects:
    bw.projects.set_current(project_name)
    print('Current project: %s  \nLocation: %s' % (bw.projects.current, bw.projects.dir))
else: 
    bw.projects.create_project(project_name)
    bw.projects.set_current(project_name)
    print('Current project: %s  \nLocation: %s' %  (bw.projects.current, bw.projects.dir))

# Check that the project was added to the directory
print(bw.projects)

Current project: lca_bark  
Location: /home/leabraud/Documents/LCAbark/projects/lca_bark.eb0fa584e58a983f7bce69d910b05036
Brightway2 projects manager with 4 objects:
	default
	lca_bark
	lcabark
	test_project
Use `projects.report()` to get a report on all projects.


## Import biosphere3

In [7]:
if 'biosphere3' in bw.databases:
        print('biosphere3 has already been imported in the project.')
else:
    bw.bw2setup()

biosphere3 has already been imported in the project.


## Import ecoinvent 3.9 consequential

In [8]:
from  bw2io import SingleOutputEcospold2Importer

ei_dir = str(str(dbdir)+'/ecoinvent 3.9_consequential_ecoSpold02/datasets')
print(ei_dir)

if 'ecoinvent_3.9_conseq' in bw.databases:
        print("ecoinvent 3.9 consequential has already been imported")
    
else:
    ei = SingleOutputEcospold2Importer(ei_dir, 'ecoinvent_3.9_conseq',use_mp=False)
    ei.apply_strategies()
    ei.statistics()
    ei.write_database()
    
bw.databases

NameError: name 'dbdir' is not defined

## Generate future versions of ecoinvent using premise

In [45]:
from premise import *

In [46]:
clear_cache()

Cache folder cleared!


In [47]:
# the values can be changed - see https://chemrxiv.org/engage/chemrxiv/article-details/63ee10cdfcfb27a31fe227d
args = {
    "range time":0, #2
    "duration":0,
    "foresight":False,
    "lead time":False,
    "capital replacement rate":False,
    "measurement": 0,
    "weighted slope start": 0.75,
    "weighted slope end": 1.00
}

In [48]:
ndb = NewDatabase(
    scenarios=[
        #{"model":"remind", "pathway":"SSP2-PkBudg500", "year":2045},
        #{"model":"remind", "pathway":"SSP2-Base", "year":2045},
        #{"model":"remind", "pathway":"SSP1-PkBudg500", "year":2045},
        {"model":"image", "pathway":"SSP2-Base", "year":2045}
    ],
    source_db="ecoinvent_3.9_conseq",
    source_version="3.9",
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=', ## DECRYPTION KEY TO BE ADDED HERE
    system_model="consequential",
    system_args=args
)

premise v.(2, 0, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

100%|██████████████████████████████████| 18856/18856 [00:00<00:00, 53000.70it/s]


Adding exchange data to activities


100%|█████████████████████████████████| 547705/547705 [01:03<00:00, 8654.04it/s]


Filling out exchange data


100%|███████████████████████████████████| 18856/18856 [00:05<00:00, 3600.99it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Correct missing flow categories for biosphere exchanges
Remove empty exchanges.
Remove uncertainty data.
- Extracting inventories
Cannot find cached inventories. Will create them now for next time...
Importing default inventories...

Extracted 1 worksheets in 1.92 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove uncertainty data.
Extracted 1 worksheets in 0.08 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove unce

/home/leabraud/miniconda3/envs/abconseq/lib/python3.11/site-packages/premise/data_collection.py:894: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_fuel_df.loc[:, "variable"] = iam_var
/home/leabraud/miniconda3/envs/abconseq/lib/python3.11/site-packages/premise/data_collection.py:898: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_fuel_df.loc[:, cols] = (
/home/leabraud/miniconda3/envs/abconseq/lib/python3.11/site-packages/premise/data_collection.py:894: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s


Summary of the electricity marginal market mixes:
+--------+--------------------+------------+----------+-----------------+---------------+-------------------------+---------------+
| Region | Measurement method | Foresight? | Duration | Avg. start year | Avg. end year | Avg. capital repl. rate | Volume change |
+--------+--------------------+------------+----------+-----------------+---------------+-------------------------+---------------+
|  BRA   |         0          |   False    |    0     |       2045      |      2124     |          -0.01          |     -0.05     |
+--------+--------------------+------------+----------+-----------------+---------------+-------------------------+---------------+
|  CAN   |         0          |   False    |    0     |       2045      |      2118     |          -0.01          |     -0.04     |
+--------+--------------------+------------+----------+-----------------+---------------+-------------------------+---------------+
|  CEU   |         0     

In [49]:
ndb.update("electricity")
ndb.update("fuels")
ndb.write_db_to_brightway()

Updating: electricity: 100%|███████████| 1/1 [02:56<00:00, 176.10s/it]


Done!



Updating: fuels: 100%|█████████████████| 1/1 [03:38<00:00, 218.77s/it]


Done!

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:21


Title: Writing activities to SQLite3 database:
  Started: 12/22/2024 23:30:59
  Finished: 12/22/2024 23:32:20
  Total time elapsed: 00:01:21
  CPU %: 100.00
  Memory %: 35.58
Created database: ecoinvent_consequential_3.9_image_SSP2-Base_2045
Generate scenario report.
Report saved under /home/leabraud/Documents/LCAbark/notebooks/export/scenario_report.
Generate change report.
Report saved under /home/leabraud/Documents/LCAbark/notebooks.


In [50]:
bw.databases

Databases dictionary with 6 object(s):
	biosphere3
	ecoinvent_3.9_conseq
	ecoinvent_consequential_3.9_image_SSP2-Base_2045
	ecoinvent_consequential_3.9_remind_SSP1-PkBudg500_2045
	ecoinvent_consequential_3.9_remind_SSP2-Base_2045
	ecoinvent_consequential_3.9_remind_SSP2-PkBudg500_2045